In [31]:
from sklearn.datasets import load_diabetes
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import torch
import nltk
import torch.nn as nn 
import json
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import os

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fjordrunner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
with open('songs_training_data.json', 'r') as fp:
    songs_training_data = json.load(fp)

In [33]:
print(len(songs_training_data))

32028


In [34]:
word_counts = Counter()

for k, v in songs_training_data.items():
    lyrics = v['lyrics']
    tokens = nltk.word_tokenize(lyrics)
    word_counts = word_counts + Counter(tokens)

shortened_word_count_info = word_counts.most_common(5000)

In [35]:
print(shortened_word_count_info)

[('i', 338181), ('you', 278589), ('the', 272055), ('and', 162608), ('to', 158267), ('a', 145738), ('it', 129066), ('me', 118227), ('my', 95859), ('in', 93998), ('not', 93416), ('is', 80117), ('of', 78730), ('that', 76538), ('do', 75242), ('your', 67830), ('on', 65874), ('we', 58942), ('all', 54444), ('are', 52359), ('be', 51942), ('for', 50632), ('love', 49944), ('am', 48900), ('will', 44336), ('know', 43278), ('no', 43144), ('so', 42889), ('but', 42062), ('have', 40850), ('with', 38531), ('like', 38441), ('just', 38086), ('this', 35900), ('what', 35307), ('got', 34356), ('oh', 33215), ('when', 33112), ('up', 32833), ('na', 31427), ('can', 29379), ('now', 28674), ('get', 28271), ('go', 28256), ('there', 26681), ('they', 26416), ('she', 26360), ('time', 26343), ('was', 26322), ('if', 25994), ('out', 25859), ("n't", 25824), ('come', 25523), ('yeah', 25335), ('one', 24984), ('down', 24765), ("'s", 23334), ('he', 23169), ('see', 22328), ('never', 21525), ('want', 21489), ('let', 21142), ('

In [36]:
# TODO: add [UNK] token
 
word_to_idx = {tup[0]: i for i, tup in enumerate(shortened_word_count_info)}
word_to_idx["<UNK>"] = len(shortened_word_count_info)

idx_to_word =  dict((v,k) for k,v in word_to_idx.items())
idx_to_word[len(shortened_word_count_info)] = "<UNK>"

In [37]:
print(len(word_to_idx))

5001


In [8]:
print(word_to_idx)

{'i': 0, 'you': 1, 'the': 2, 'and': 3, 'to': 4, 'a': 5, 'it': 6, 'me': 7, 'not': 8, 'my': 9, 'in': 10, 'is': 11, 'of': 12, 'do': 13, 'that': 14, 'your': 15, 'on': 16, 'we': 17, 'all': 18, 'are': 19, 'be': 20, 'love': 21, 'for': 22, 'am': 23, 'will': 24, 'know': 25, 'so': 26, 'have': 27, 'but': 28, 'no': 29, 'like': 30, 'with': 31, 'just': 32, 'what': 33, 'this': 34, 'got': 35, 'oh': 36, 'when': 37, 'up': 38, 'na': 39, 'can': 40, 'now': 41, 'get': 42, 'go': 43, 'she': 44, 'there': 45, 'was': 46, 'out': 47, 'time': 48, 'if': 49, 'they': 50, 'yeah': 51, 'come': 52, 'down': 53, 'one': 54, "n't": 55, 'he': 56, 'see': 57, 'want': 58, 'let': 59, 'never': 60, "'s": 61, 'say': 62, 'way': 63, 'from': 64, 'make': 65, 'feel': 66, 'ca': 67, 'would': 68, 'take': 69, 'at': 70, 'back': 71, 'as': 72, "'m": 73, 'babi': 74, 'her': 75, 'gon': 76, "'": 77, 'day': 78, 'need': 79, 'could': 80, 'away': 81, 'here': 82, 'how': 83, 'heart': 84, 'been': 85, 'life': 86, 'right': 87, 'night': 88, 'look': 89, 'by': 

In [38]:
max_length = 0
for k, v in songs_training_data.items():
    lyrics = v['lyrics']
    tokens = nltk.word_tokenize(lyrics)
    # TODO: include the [UNK] token for words not seen in the corpus, (make and if else statement)
    numerical_form = []
    for word in tokens:
      if word in word_to_idx:
        numerical_form.append(word_to_idx[word])
      else:
        numerical_form.append(word_to_idx["<UNK>"])

        
    # numerical_form = [word_to_idx[word] for word in tokens if word in word_to_idx ]
    # print(len(numerical_form))

    songs_training_data[k]['tensor'] = torch.tensor(numerical_form)
    max_length = max(max_length, songs_training_data[k]['tensor'].size()[0])

In [39]:
print(len(shortened_word_count_info))

5000


In [40]:
print(len(songs_training_data))

32028


In [41]:
lyrics = [songs_training_data[k]['tensor'] for k in songs_training_data]
print(len(lyrics))

lyrics = pad_sequence(lyrics, batch_first= True, padding_value=len(shortened_word_count_info))

tags = np.array([songs_training_data[k]['tag_list'] for k in songs_training_data])

32028


In [42]:
tags_updated = tags.argmax(1)
tags_updated = np.where(tags_updated > 12, tags_updated - 1, tags_updated)
tags_updated.shape

(32028,)

In [43]:
class trainingdataset(Dataset):
    def __init__(self, lyrics, tags):
        self.x = torch.tensor(lyrics)
        self.y = torch.tensor(tags)
        self.length = self.x.shape[0]

    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return self.length


training_dataset = trainingdataset(lyrics, tags_updated)

/Users/fjordrunner/cs4300-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
class lyric_based_training_dataset(Dataset):
    def __init__(self, lyrics):
        self.x = torch.tensor(lyrics)
        self.length = self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index]
    
    def __len__(self):
        return self.length

training_dataset = lyric_based_training_dataset(lyrics)

/Users/fjordrunner/cs4300-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
len(training_dataset), training_dataset[0]

(32028, tensor([ 893,    0,  265,  ..., 5000, 5000, 5000]))

In [46]:
dataloader = DataLoader(dataset=training_dataset, shuffle=True, batch_size=1)

In [47]:
from itertools import chain
from sklearn.utils import class_weight

train_labels = list(tags_updated)
label_ids = np.unique(train_labels)
print(len(label_ids))

class_weights = class_weight.compute_class_weight('balanced', classes=label_ids, y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights, len(class_weights)

13


(tensor([0.4856, 0.9490, 6.8246, 0.7632, 1.0806, 0.8715, 1.4859, 1.6414, 0.6530,
         1.4200, 2.2541, 1.3310, 0.6080]), 13)

In [48]:
print(label_ids)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [49]:
print(len(shortened_word_count_info))

5000


In [50]:
class net(nn.Module):
  def __init__(self, embedding_dim = 500, hidden_dim=250, hidden2_dim = 100, padding_idx=len(shortened_word_count_info),
               output_size=len(class_weights)):
    super(net,self).__init__()
    self.l1 = nn.Embedding(num_embeddings=len(shortened_word_count_info) + 1, 
                           embedding_dim=embedding_dim, padding_idx=padding_idx)
    nn.init.xavier_uniform_(self.l1.weight.data)
    self.l1.weight.data[padding_idx] = torch.zeros(embedding_dim)

    self.l2 = nn.Linear(embedding_dim, hidden_dim)
    nn.init.xavier_uniform_(self.l2.weight.data)

    self.relu = nn.GELU()
    self.l3 = nn.Linear(hidden_dim, hidden2_dim)

    self.l4 = nn.Linear(hidden2_dim, output_size)

  def forward(self, x):
    # x: (b, L)
    # o: (b, L, e)
    output1 = self.l1(x) 
    # o: (b, e)
    # print(output.shape)
    # 500-dimensional
    output2 = torch.mean(output1, 1)
    # o: (b, h)
    #250-dimensional
    output3 = self.relu(self.l2(output2))
    # o: (b, o)

    output4 = self.relu(self.l3(output3))

    output = self.l4(output4)
    # (b, 9)

    return output, output3, output4

In [51]:
model = torch.load('model.pt', map_location=torch.device('cpu'))

In [52]:
mat_100 = []


for x_train in dataloader:
  _, __out, _100_output =  model(x_train.cpu())
  mat_100.append(_100_output)

In [53]:
for i, tens in enumerate(mat_100):
    mat_100[i] = tens.tolist()

In [54]:
# for i, tens in enumerate(mat_250):
#     mat_250[i] = tens.tolist()

In [55]:
with open('./100_lyric_embeddngs.json', 'w') as fp:
    json.dump(mat_100, fp)

In [56]:
# with open('./250_lyric_embeddngs.json', 'w') as fp:
#     json.dump(mat_250, fp)

In [57]:
print(len(mat_100))

32028


In [67]:
print(len(mat_250))

32028
